In [47]:
# includes -
include("./code/Include.jl");

##### Load the stoichiometric matrix 

In [6]:
stoichiometric_matrix = readdlm("./code/network/Network.csv",',')

8×9 Array{Float64,2}:
 -1.0   0.0   0.0   0.0   0.0   0.0   1.0   0.0   0.0
  1.0  -1.0  -1.0   1.0   0.0   0.0   0.0   0.0   0.0
  0.0   1.0   0.0   0.0   0.0   0.0   0.0  -1.0   0.0
  0.0   0.0   1.0  -1.0  -1.0   1.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   1.0  -1.0   0.0   0.0  -1.0
  0.0   0.0   0.0   0.0   0.0   0.0  -1.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0

In [8]:
(number_of_metabolites, number_of_reactions) = size(stoichiometric_matrix)

(8, 9)

##### Compute the metabolite ($A_{m}$) and reaction ($A_{v}$) Adjacency matrices

In [18]:
binary_stoichiometric_matrix = make_binary_stoichiometric_matrix(stoichiometric_matrix)

8×9 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0

In [19]:
A_m = binary_stoichiometric_matrix*transpose(binary_stoichiometric_matrix)

8×8 Array{Float64,2}:
 2.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0
 1.0  4.0  1.0  2.0  0.0  0.0  0.0  0.0
 0.0  1.0  2.0  0.0  0.0  0.0  1.0  0.0
 0.0  2.0  0.0  4.0  2.0  0.0  0.0  0.0
 0.0  0.0  0.0  2.0  3.0  0.0  0.0  1.0
 1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0

In [20]:
A_v = transpose(binary_stoichiometric_matrix)*binary_stoichiometric_matrix

9×9 Array{Float64,2}:
 2.0  1.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0
 1.0  2.0  1.0  1.0  0.0  0.0  0.0  1.0  0.0
 1.0  1.0  2.0  2.0  1.0  1.0  0.0  0.0  0.0
 1.0  1.0  2.0  2.0  1.0  1.0  0.0  0.0  0.0
 0.0  0.0  1.0  1.0  2.0  2.0  0.0  0.0  1.0
 0.0  0.0  1.0  1.0  2.0  2.0  0.0  0.0  1.0
 1.0  0.0  0.0  0.0  0.0  0.0  2.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  2.0  0.0
 0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  2.0

#### Singular value decomposition (SVD) of the stoichiometric matrix

The [SVD of a matrix](https://www.youtube.com/watch?v=mBcLRGuAFUk) is a method to decompose an array $A$ into three arrays:
\begin{equation}
A = U\Sigma{V}^{T}
\end{equation}which is the product of rotation $\times$ stretch $\times$ rotation operations. The SVD of the stoichiometric array has been used to get some additional structural insight into a biological reaction network (see the [Palsson study](https://www.ncbi.nlm.nih.gov/pubmed/12900206) or the [Palsson SVD stoichiometric analysis video](https://www.youtube.com/watch?v=8W7RBt_iCYM&feature=youtu.be))

In [23]:
(U,S,V) = svd(stoichiometric_matrix);

##### Check accuracy:

In [31]:
error_term = norm(stoichiometric_matrix - U*diagm(S)*transpose(V));

In [32]:
error_term

8.713796557141027e-15

In [33]:
diagm(S)

8×8 Array{Float64,2}:
 2.60695  0.0      0.0      0.0      0.0      0.0       0.0       0.0       
 0.0      2.04381  0.0      0.0      0.0      0.0       0.0       0.0       
 0.0      0.0      1.61803  0.0      0.0      0.0       0.0       0.0       
 0.0      0.0      0.0      1.53088  0.0      0.0       0.0       0.0       
 0.0      0.0      0.0      0.0      1.14812  0.0       0.0       0.0       
 0.0      0.0      0.0      0.0      0.0      0.618034  0.0       0.0       
 0.0      0.0      0.0      0.0      0.0      0.0       0.604051  0.0       
 0.0      0.0      0.0      0.0      0.0      0.0       0.0       4.5905e-17

In [34]:
rank(stoichiometric_matrix)

7

In [36]:
fraction = (S./sum(S))*100.0

8-element Array{Float64,1}:
 25.63401371619153     
 20.096743282436805    
 15.910061691856608    
 15.05308187167751     
 11.289383499874889    
  6.077102803181567    
  5.939613134781095    
  4.513817863863056e-16

In [49]:
FM = fractional_reconstruction_of_stoichiometric_matrix(stoichiometric_matrix, U,S,V);

In [50]:
FM[1]

8×9 Array{Float64,2}:
 -0.0897453   0.0897453   0.160989   -0.160989   …  -0.0187118    0.0564329 
  0.414951   -0.414951   -0.744355    0.744355       0.086517    -0.260926  
 -0.0897453   0.0897453   0.160989   -0.160989      -0.0187118    0.0564329 
 -0.490393    0.490393    0.879686   -0.879686      -0.102247     0.308365  
  0.270662   -0.270662   -0.485525    0.485525       0.0564329   -0.170196  
  0.0154835  -0.0154835  -0.027775    0.027775   …   0.0032283   -0.00973623
  0.0154835  -0.0154835  -0.027775    0.027775       0.0032283   -0.00973623
 -0.0466967   0.0466967   0.0837664  -0.0837664     -0.00973623   0.0293635 

In [51]:
FM[2]

8×9 Array{Float64,2}:
 -0.362744   0.362744   0.258977   …   0.144103   -0.144103   -0.186375 
  0.923393  -0.923393  -0.926852      -0.32005     0.32005     0.191288 
 -0.362744   0.362744   0.258977       0.144103   -0.144103   -0.186375 
 -0.262436   0.262436   0.797865      -0.0024562   0.0024562   0.511112 
 -0.257974   0.257974  -0.29578        0.186375   -0.186375   -0.640371 
  0.101408  -0.101408  -0.0586163  …  -0.0426945   0.0426945   0.0666863
  0.101408  -0.101408  -0.0586163     -0.0426945   0.0426945   0.0666863
  0.119689  -0.119689   0.024045      -0.0666863   0.0666863   0.177349 

In [52]:
FM[end]

8×9 Array{Float64,2}:
 -1.0           8.39606e-16  -3.1225e-16   …  -1.08941e-15  -3.747e-16  
  1.0          -1.0          -1.0              2.37787e-15  -1.23599e-16
  1.13798e-15   1.0          -4.71845e-16     -1.0           1.38778e-17
 -3.1225e-16   -6.59195e-16   1.0              1.09635e-15   2.77556e-17
  1.42941e-15   8.46545e-16   4.30211e-16     -2.51188e-15  -1.0        
 -6.38378e-16  -3.05311e-16  -5.55112e-17  …   5.96745e-16   6.93889e-17
 -6.245e-16    -1.80411e-16  -7.21645e-16      1.0          -1.38778e-16
 -2.498e-16    -2.22045e-16   4.71845e-16     -5.55112e-17   1.0        